In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# CITIREA DATELOR DIN FISIERELE TEXT

In [ ]:
train_sample_stuff = pd.read_csv('data/train_samples.txt', sep='	', header=None)
# retin doar textele intr-un array (fara id-ul de la inceput)
train_samples = train_sample_stuff[1]

train_label_stuff = pd.read_csv('data/train_labels.txt', sep = '\t', header=None)
# retin doar label-urile intr-un array (fara id-ul de la inceput)
train_labels = train_label_stuff[1]

In [ ]:
validation_sample_stuff = pd.read_csv('data/validation_samples.txt', sep='	', header=None)
# retin doar textele intr-un array (fara id-ul de la inceput)
validation_samples = validation_sample_stuff[1]

validation_label_stuff = pd.read_csv('data/validation_labels.txt', sep='	', header=None)
# retin doar label-urile intr-un array (fara id-ul de la inceput)
validation_labels = validation_label_stuff[1]

In [ ]:
test_sample_stuff = pd.read_csv('data/test_samples.txt', sep='	', header=None)
test_samples = test_sample_stuff[1]
test_list = [word.split() for word in test_samples]

In [ ]:
# ONE HOT ENCODING
# transform fiecare label intr-un array de 3 'biti'
# bitul de pe pozitia 1 reprezinta labelul 1 s.a.m.d.

In [ ]:
# array-ul este intai initializat cu zerouri; 10000 de linii cate texte sunt in setul de validare si 3 coloane cate sunt label-urile
train_labels_encoded = np.zeros((10000, 3), dtype=int)

# apoi pentru fiecare text, formatez label-ul setand bitul 1 in functie de pozitie
for i in range(0,5000):
    tl = train_labels[i]
    if tl==1:
        train_labels_encoded[i, 0] = 1
    if tl==2:
        train_labels_encoded[i, 1] = 1
    if tl==3:
        train_labels_encoded[i, 2] = 1

In [ ]:
# array-ul este intai initializat cu zerouri; 5000 de linii cate texte sunt in setul de validare si 3 coloane cate sunt label-urile
validation_labels_encoded = np.zeros((5000, 3), dtype=int)

# apoi pentru fiecare text, formatez label-ul setand bitul 1 in functie de pozitie
for i in range(0,5000):
    tl = validation_labels[i]
    if tl==1:
        validation_labels_encoded[i, 0] = 1
    if tl==2:
        validation_labels_encoded[i, 1] = 1
    if tl==3:
        validation_labels_encoded[i, 2] = 1

In [ ]:
# PREPROCESAREA TEXTULUI CU TFID
# in functie de probabilitatea de aparitie a unui cuvant in texte, le ignora pe cele cu aceasta metrica foarte mica
from sklearn.feature_extraction.text import TfidfVectorizer

# desparte cuvintele dupa spatii
tfidf_vec = TfidfVectorizer(token_pattern = r'[^\s]+')
# invatare vocabular din setul de train si construire a matricii de probabilitati de aparitie a fiecarui cuvant din vocabular
X_train_TF = tfidf_vec.fit_transform(train_samples) 
# foloseste si datele de validare si testare pentru a construi matricea de pbb 
X_val_TF = tfidf_vec.transform(validation_samples) 
X_test_TF = tfidf_vec.transform(test_samples) 

# returneaza un tuplu (document_id, token_id) si probabilitatea cu care apare respectivul cuvant in dictionar

In [ ]:
# DEFINIREA MODELULUI

from keras.models import Sequential
from keras import layers

# dimensiunea inputului pentru primul strat din reteaua neurala
dimensiune_input = X_train_TF.shape[1]

modelSeq = Sequential()
# primul strat din retea: dimensiune_output, dimensiune_input, functie_de_activare
modelSeq.add(layers.Dense(100, input_dim=input_dim, activation='relu'))
# al doilea strat din retea: dimensiune_output, functie_de_activare 
modelSeq.add(layers.Dense(3, activation='softmax'))
# dimensiunea input-ului pentru acesta este dimensiunea output-ului layer-ului anterior, adica 100

In [ ]:
# CONFIGURAREA MODELULUI PENTRU ANTRENARE

# functie de calculare a loss-ului, algoritm de optimizare a loss-ului, metrica de calculare a performantei
modelSeq.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

In [ ]:
# ANTRENAREA MODELULUI

# inputul sunt datele de antrenare, nr de iteratii facute pe input pentru antrenare, fara detalii, setul pe care sa se faca evaluarea scorului este cel 
# de validare, batch_size reprezinta numarul de texte dupa care sa actualizeze gradientul daca acesta nu s-a imbunatatitit
modelSeq.fit(X_train_TF, train_labels_encoded,
                    epochs=100,
                    verbose=False,
                    validation_data=(X_val_TF, validation_labels_encoded),
                    batch_size=10)

In [ ]:
# EVALUAREA ACURATETII

# texte_validare, labels_validare, fara_detalii
loss, accuracy = model.evaluate(X_val_TF, validation_labels_encoded, verbose=False)
# print("Validation Accuracy:  {:.4f}".format(accuracy))

In [ ]:
# PREZICEREA LABEL-URILOR PENTRU SETUL DE TESTARE

predicted_test_labels = model.predict(X_test_TF)

# este nevoie de rotunjirea label-urilor din cauza ca TfidVectorizer a returnat probabilitati pentru label-urile prezise
predicted_test_labels_rounded = np.around(predicted_test_labels)

In [ ]:
# SCRIEREA LABEL-URILOR IN FISIERUL TEXT DE SUBMISIE

# x este lista pe care o voi scrie in fisierul de submisie
x = [['id','label']]
test_ids = test_sample_stuff[0]

for i in range(len(test_sample_stuff)):
    row = []
    predicted_array_of_labels = predicted_test_labels_rounded[i]
    if predicted_array_of_labels[0]:
        predicted_label = 1
    if predicted_array_of_labels[1]:
        predicted_label = 2
    if predicted_array_of_labels[2]:
        predicted_label = 3
    row.append(test_ids[i])
    row.append(predicted_label)
    x.append(row)

np.savetxt('sample_submission.txt', x, delimiter=',', newline='\n', fmt='%s')